# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# План работы:

1) [Изучить общую информацию.](#id_1)

2) [Обучение и проверка модели.](#id_2)

3) [Подготовка к расчету прибыли.](#id_3)

4) [Расчет прибыли и рисков.](#id_4)

5) [Общий вывод.](#id_5)

<a id='id_1'></a>
# 1. Загрузка и подготовка данных

In [1]:
# импортируем все необходимые для нас библиотеки
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy import stats as st
data1 = pd.read_csv('/datasets/geo_data_0.csv')
data2 = pd.read_csv('/datasets/geo_data_1.csv')
data3 = pd.read_csv('/datasets/geo_data_2.csv')

# ознакамливаемся с данными
data1.info()
data1.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [2]:
data2.head(5)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [3]:
data2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [4]:
data3.info()
data3.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Теперь у нас есть 3 датафрейма, для каждого региона свой.
Все они однотипны - имеют 100000 строк и 5 столбцов. Столбцы f0, f1, f2 - являются признаками, столбец product - целевым признаком. Признаки имеют разный мастштаб, а значит, для более корректного предсказания моделью - проведем масштабирование.

In [5]:
# Проведем масштабирование для первого региона
to_scaler = ['f0', 'f1', 'f2']
scaler1 = StandardScaler()
scaler1.fit(data1[to_scaler])
data1[to_scaler] = scaler1.transform(data1[to_scaler])
data1.describe()

,f0,f1,f2,product
count,1.000000e+05,1.000000e+05,1.000000e+05,100000.000000
mean,2.287948e-17,-5.044853e-17,9.608314e-17,92.500000
std,1.000005e+00,1.000005e+00,1.000005e+00,44.288691
min,-2.189681e+00,-2.177430e+00,-4.491975e+00,0.000000
25%,-6.572397e-01,-8.941262e-01,-6.818784e-01,56.497507
50%,2.226500e-03,2.166008e-04,4.101360e-03,91.849972
75%,6.574259e-01,8.930937e-01,6.811217e-01,128.564089
max,2.135642e+00,2.168043e+00,4.156460e+00,185.364347


In [6]:
# Аналогично для второго и третьего
scaler2 = StandardScaler()
scaler2.fit(data2[to_scaler])
data2[to_scaler] = scaler2.transform(data2[to_scaler])
data2.describe()

,f0,f1,f2,product
count,1.000000e+05,1.000000e+05,1.000000e+05,100000.000000
mean,-7.105427e-18,1.854517e-17,1.444533e-16,68.825000
std,1.000005e+00,1.000005e+00,1.000005e+00,45.944423
min,-3.652831e+00,-4.211458e+00,-1.474959e+00,0.000000
25%,-8.297949e-01,-6.780293e-01,-8.772907e-01,26.953261
50%,1.311571e-03,-3.240857e-03,-2.835602e-01,57.085625
75%,8.342420e-01,6.765366e-01,8.836555e-01,107.813044
max,3.154229e+00,4.595967e+00,1.482293e+00,137.945408


In [7]:
scaler3 = StandardScaler()
scaler3.fit(data3[to_scaler])
data3[to_scaler] = scaler3.transform(data3[to_scaler])
data3.describe()

,f0,f1,f2,product
count,1.000000e+05,1.000000e+05,1.000000e+05,100000.000000
mean,2.241762e-17,-1.435296e-17,-1.391953e-16,95.000000
std,1.000005e+00,1.000005e+00,1.000005e+00,44.749921
min,-5.058801e+00,-4.092640e+00,-4.164608e+00,0.000000
25%,-6.722210e-01,-6.777239e-01,-6.808172e-01,59.450441
50%,4.272944e-03,-4.277388e-03,-3.135732e-03,94.925613
75%,6.677185e-01,6.736897e-01,6.804996e-01,130.595027
max,4.177880e+00,4.534699e+00,4.100928e+00,190.029838


## Вывод по п.1:
Итак, данные прочитаны и масштабированы, все необходимые библиотеки импортированы, пора переходить к моделям

<a id='id_2'></a>
# 2. Обучение и проверка модели

___Для удобства каждый регион имеет свой порядковый номер (1-3), соответственно, модели и переменные для каждого региона будут использоваться с тем же порядковым номером в названии.___

## 2.1 Регион 1

In [8]:
# Создадим обучающую и валидационные выборки с соотношением 75 к 25.
data1_train, data1_valid = train_test_split(data1, test_size=0.25, random_state=123)
data1_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 31450 to 15725
Data columns (total 5 columns):
id         75000 non-null object
f0         75000 non-null float64
f1         75000 non-null float64
f2         75000 non-null float64
product    75000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.4+ MB


In [9]:
# Обучим модель на тренировочной выборке. Из условий заданий - пользуемся линейной регрессией.

# Отдельно сохраним признаки и целевой признак
features_train1 = data1_train.drop(['id', 'product'], axis=1)
target_train1 = data1_train['product']
features_valid1 = data1_valid.drop(['id', 'product'], axis=1)
target_valid1 = data1_valid['product']

# Создаем и обучаем модель
model1 = LinearRegression()
model1.fit(features_train1, target_train1) 
predict1 = model1.predict(features_valid1) 

In [10]:
# Посчитаем средние размеры предсказанного сырья
predict1.mean()

92.54936189116306

In [11]:
# Для проверки качества модели так же напечатаем средние размеры сырья по валидационной сборке
target_valid1.mean()

92.85062391123445

In [12]:
# Средние размеры предсказанного сырья и фактического примерно равны, что говорит о качественной работе модели.
# Так же посчитаем среднеквадратическую ошибку (RMSE) 
rmse1 = mean_squared_error(target_valid1, predict1)**0.5
rmse1

37.64786282376176

Среднее значение предсказанного сырья очень близко к среднему значению сырья. 
Среднеквадратическая ошибка = 37.65, а значит в среднем предсказанное количество сырья отличается от фактического практически на 40%, что достаточно много.

## 2.2 Регион 2

In [13]:
# Тут мы делаем аналогично региону 1
# Создадим тренировочную и валидационную выборку, отделим признаки и целевой признак
data2_train, data2_valid = train_test_split(data2, test_size=0.25, random_state=123)
features_train2 = data2_train.drop(['id', 'product'], axis=1)
target_train2 = data2_train['product']
features_valid2 = data2_valid.drop(['id', 'product'], axis=1)
target_valid2 = data2_valid['product']

In [14]:
# Создаем модель
model2 = LinearRegression()
model2.fit(features_train2, target_train2) 
predict2 = model2.predict(features_valid2) 
predict2.mean()

69.28001860653976

In [15]:
target_valid2.mean()

69.27371236077902

In [16]:
rmse2 = mean_squared_error(target_valid2, predict2)**0.5
rmse2

0.8954139804944304

Аналогично региону 1 средние значения практически не различаются, однако среднеквадратиеская ошибка менее 1, что говорит о качестве обученной модели по региону 2.

## 2.3 Регион 3

In [17]:
# Действуем по аналогии с предыдущими пунктами
data3_train, data3_valid = train_test_split(data3, test_size=0.25, random_state=123)
features_train3 = data3_train.drop(['id', 'product'], axis=1)
target_train3 = data3_train['product']
features_valid3 = data3_valid.drop(['id', 'product'], axis=1)
target_valid3 = data3_valid['product']

model3 = LinearRegression()
model3.fit(features_train3, target_train3) 
predict3 = model3.predict(features_valid3) 
predict3.mean()

95.09859933591373

In [18]:
target_valid3.mean()

94.87348818660215

In [19]:
rmse3 = mean_squared_error(target_valid3, predict3)**0.5
rmse3

40.12803006598514

Опять-таки, средние значения сырья по предсказанию и факту отличаются не сильно, но среднеквадратическая ошибка у региона 3 - самая большая (в среднем предсказанный вариант отличается от фактического на 43%). 

## Вывод по п.2:

Регион 1:    

    - среднее прогнозируемое значение составляет 92.55
    - среднее значение по факту составляет 92.85
    - среднеквадратическая ошибка 37.65
    
Регион 2:    

    - среднее прогнозируемое значение составляет 69.28
    - среднее значение по факту составляет 69.27
    - среднеквадратическая ошибка 0.89
    
Регион 3:    

    - среднее прогнозируемое значение составляет 95.1
    - среднее значение по факту составляет 94.87
    - среднеквадратическая ошибка 40.13
    
Среднее прогнозируемое значение во всех регионах очень близко к фактическому среднему значению. Но в то время как в регионе 1 и регионе 3 значение составило более 90, в регионе 2 - 69, что значимо меньше других.

По среднеквадратической ошибке: в регионе 1 и регионе 3 её значение составило близко к 40, в то время как в регионе 2 менее 1, что говорит и минимальных расхождениях при прогнозе в регионе 2.

<a id='id_3'></a>
# 3. Подготовка к расчёту прибыли

Как нам уже известно из условий - всего исследуются 500 точек, из которых будут использоваться всего 200.
Бюджет на разработку скважин 10 млрд рублей. 
Доход с одной скважины - 450 т.р. за одну единицу. Сохраним эти данные в переменных

In [20]:
# Для удобства будет считать все цифры в тысячах рублей
budget = 10000000
price = 450
well = 200
# Посчитаем сколько бюджета выделяется на одну скважину
budget_to_well = budget/well
budget_to_well

50000.0

In [21]:
# Теперь посчитаем минимальное количество сырья в скважине, чтобы не допустить убытков
mean_revenue = budget_to_well/price
mean_revenue

111.11111111111111

In [22]:
# Посчитаем средние количества объемов сырья в каждом из регионов
data1['product'].mean()

92.50000000000001

In [23]:
data2['product'].mean()

68.82500000000002

In [24]:
data3['product'].mean()

95.00000000000004

### Вывод по пункту 3

В среднем на одну скважину выделяется 50000 т.р.

Минимальное количество сырья для безубыточности 111.11 тысяч баррелей.

В среднем, в регионах у нас 92.5, 68.82, 95 тысяч баррелей сырья. Регион 1 и регион 3 имеют высокие значения, в то время как регион 2 имеет значение значимо меньше остальных. Но, с учетом того, что у нас будут использоваться всего 200 скважин, в то время как в регионе исследовалось 100000 точек - сбрасывать со счетов нельзя ни одну из скважин.

<a id='id_4'></a>
# 4. Расчёт прибыли и рисков 

In [25]:
# Напишем функцию для расчета прибыли по предсказаниям модели и скважинам
def revenue(target, predict, count):
    sum_product = 0
    # Отсортируем предсказания по максимальным значениям
    predict_sort = predict.sort_values(ascending=False) 
    # Отсортируем скважины, убрав непопвашие в список (count)
    target_sort = target[predict_sort.index][:count]
    # Сложим получившиеся результаты
    for i in target_sort:
        sum_product += i  
    # Посчитаем выручку
    return 450 * sum_product

In [26]:
# У предсказаний (predict) индексы от 0 до 24999, в то время как у целевых показателей на валидацонной выборке индексы находятся
# в диапозоне от 0 до 99999. Для того, чтобы в последствии не возникало ошибок - обнулим индексы у валидационной выборки по
# целевым показателям
target_valid1 = target_valid1.reset_index(drop = True)
target_valid2 = target_valid2.reset_index(drop = True)
target_valid3 = target_valid3.reset_index(drop = True)

# Так же переведем предсказания в формат датафрейма, чтобы потом можно было их сортировать
predict1 = pd.Series(predict1)
predict2 = pd.Series(predict2)
predict3 = pd.Series(predict3)

## 4.1 Регион 1

In [27]:
# Используем технику Bootstrap с 1000 выборок
values1 = []

state = np.random.RandomState(12345)

for i in range(1000):
    target_subsample1 = target_valid1.sample(500, random_state=state, replace=True)
    predict_subsample1 = predict1[target_subsample1.index]
    values1.append(revenue(target_subsample1, predict_subsample1, 200))
    
values1 = pd.Series(values1)

# Посчитаем среднее значение прибыли
mean1 = values1.mean()
mean1

10505264.703706196

In [28]:
# Посчитаем 95% доверительный интервал
interval1 = st.t.interval(0.95, len(values1)-1, values1.mean(), values1.sem())
interval1

(10488019.000641398, 10522510.406770993)

In [29]:
# Посчитаем общее количество результатов менее 10000000 (убыточные результаты)
unprofitable1 = values1[values1 < budget]
unprofitable1.count()

26

### Вывод по региону 1:

Среднее значение прибыли составляет 10505264.70.

Доверительный интервал составляет 10488019 - 10522510.41

Из 1000 вариантов 26 убыточных, а значит риск убытков равен 2,6%

## 4.2 Регион 2

In [30]:
# Проводим аналогичные рассчеты для региона 2
# Используем технику Bootstrap с 1000 выборок
values2 = []

for i in range(1000):
    target_subsample2 = target_valid2.sample(500, random_state=state, replace=True)
    predict_subsample2 = predict2[target_subsample2.index]
    values2.append(revenue(target_subsample2, predict_subsample2, 200))
    
values2 = pd.Series(values2)

# Посчитаем среднее значение прибыли
mean2 = values2.mean()
mean2

10534110.250377888

In [31]:
# Посчитаем 95% доверительный интервал
interval2 = st.t.interval(0.95, len(values2)-1, values2.mean(), values2.sem())
interval2

(10521143.856958617, 10547076.64379716)

In [32]:
# Посчитаем общее количество результатов менее 10000000 (убыточные результаты)
unprofitable2 = values2[values2 < budget]
unprofitable2.count()

2

## Вывод по региону 2:

Среднее значение прибыли составляет 10534110.36.

Доверительный интервал составляет 10521143.86 - 10547076.64

Из 1000 вариантов 2 убыточных, а значит риск убытков равен 0,2%

## 4.3 Регион 3

In [33]:
# Проводим аналогичные рассчеты для региона 3
# Используем технику Bootstrap с 1000 выборок
values3 = []

for i in range(1000):
    target_subsample3 = target_valid3.sample(500, random_state=state, replace=True)
    predict_subsample3 = predict3[target_subsample3.index]
    values3.append(revenue(target_subsample3, predict_subsample3, 200))
    
values3 = pd.Series(values3)

# Посчитаем среднее значение прибыли
mean3 = values3.mean()
mean3

10370652.012026329

In [34]:
# Посчитаем 95% доверительный интервал
interval3 = st.t.interval(0.95, len(values3)-1, values3.mean(), values3.sem())
interval3

(10353220.36433051, 10388083.659722148)

In [35]:
# Посчитаем общее количество результатов менее 10000000 (убыточные результаты)
unprofitable3 = values3[values3 < budget]
unprofitable3.count()

100

## Вывод по региону 3:

Среднее значение прибыли составляет 10372491.56

Доверительный интервал составляет 10355102.25 - 10389880.88

Из 1000 вариантов 99 убыточных, а значит риск убытков практически равен 10%.

<a id='id_5'></a>
# 5. Общий вывод:

По условиям задачи нужно найти регион с вероятностью убытков меньше 2.5%. Из всех регионов нам подходит номер 2, в котором риск убытков составляет 0.2%. Следует отметить, что регион 1 максимально близок к показателю (риск убытков 2.6%). Но средняя выручка в регионе 2 больше чем в регионе 1, а значит и по этому параметру регион 2 нам подходит больше. Регион 3 проигрывает как по средней выручке, так и по риску убытков (около 10%, очень высокий показатель). 

___На основании вышеперечисленного, рекомендуется бурить новую скважину в регионе номер 2.___

## Исправления

In [77]:
# РЕГИОН 1
# Создадим такую функцию, которая одновременно будет и считать общую выручку, и создавать списки со скважинами, которые приносят
# прибыль и которые работают в убыток, для этого нам понадобятся 2 новых списка
# для каждого региона создадим свою функцию со своими номерами переменных
profit_values1 = []
unprofit_values1 = []
prof_values1 = []
unprof_values1 = []

def revenue1(target, predict, count):
    sum_product = 0
    predict_sort = predict.sort_values(ascending=False) 
    target_sort = target[predict_sort.index][:count]
    for i in target_sort:
        i = i*price - budget_to_well
        sum_product += i
        if i < 0:
            unprof_values1.append(i)
        else:
            prof_values1.append(i)
    if sum_product < 0:
        unprofit_values1.append(i)
    else:
        profit_values1.append(i)
    
    return sum_product

# Произведем все необходимые расчеты

values1 = []

state = np.random.RandomState(12345)

for i in range(1000):
    target_subsample1 = target_valid1.sample(500, random_state=state, replace=True)
    predict_subsample1 = predict1[target_subsample1.index]
    values1.append(revenue1(target_subsample1, predict_subsample1, 200))
    
values1 = pd.Series(values1)

# Посчитаем среднее значение прибыли
mean1 = values1.mean()
mean1

505264.70370619517

In [74]:
# Посчитаем, сколько точек с отрицательной выручкой
unprofit_values1 = pd.Series(unprofit_values1)
unprofit_values1.count()

26

In [75]:
# А теперь посчитаем положительную выручку
profit_values1 = pd.Series(profit_values1)
profit_values1.count()

974

In [76]:
confidence_interval = (values1.quantile(0.025), values1.quantile(0.975))
confidence_interval

(-1086.050099346823, 1062107.2493602617)

In [78]:
# Посчитаем сколько скважин было с положительной выручкой
prof_values1 = pd.Series(prof_values1)
prof_values1.count()

118357

In [79]:
# Посчитаем сколько скважин было с отрицательной выручкой
unprof_values1 = pd.Series(unprof_values1)
unprof_values1.count()

81643

In [92]:
# Регион 2
profit_values2 = []
unprofit_values2 = []
prof_values2 = []
unprof_values2 = []

def revenue2(target, predict, count):
    sum_product = 0
    predict_sort = predict.sort_values(ascending=False) 
    target_sort = target[predict_sort.index][:count]
    for i in target_sort:
        i = i*price - budget_to_well
        sum_product += i
        if i < 0:
            unprof_values2.append(i)
        else:
            prof_values2.append(i)
    if sum_product < 0:
        unprofit_values2.append(i)
    else:
        profit_values2.append(i)
    
    return sum_product

# Произведем все необходимые расчеты

values2 = []

for i in range(1000):
    target_subsample2 = target_valid2.sample(500, random_state=state, replace=True)
    predict_subsample2 = predict2[target_subsample2.index]
    values2.append(revenue2(target_subsample2, predict_subsample2, 200))
    
values2 = pd.Series(values2)

# Посчитаем среднее значение прибыли
mean2 = values2.mean()
mean2

525000.1518155917

In [93]:
# Посчитаем, сколько точек с отрицательной выручкой
unprofit_values2 = pd.Series(unprofit_values2)
unprofit_values2.count()

9

In [94]:
# А теперь посчитаем положительную выручку
profit_values2 = pd.Series(profit_values2)
profit_values2.count()

991

In [95]:
confidence_interval2 = (values2.quantile(0.025), values2.quantile(0.975))
confidence_interval2

(88797.38692418336, 964959.7866292426)

In [96]:
prof_values2 = pd.Series(prof_values2)
prof_values2.count()

85499

In [97]:
unprof_values2 = pd.Series(unprof_values2)
unprof_values2.count()

114501

In [98]:
# Регион 3
profit_values3 = []
unprofit_values3 = []
unprof_values3 = []
prof_values3 = []
def revenue3(target, predict, count):
    sum_product = 0
    predict_sort = predict.sort_values(ascending=False) 
    target_sort = target[predict_sort.index][:count]
    for i in target_sort:
        i = i*price - budget_to_well
        sum_product += i
        if i < 0:
            unprof_values3.append(i)
        else:
            prof_values3.append(i)
    if sum_product < 0:
        unprofit_values3.append(i)
    else:
        profit_values3.append(i)
    return sum_product

values3 = []

for i in range(1000):
    target_subsample3 = target_valid3.sample(500, random_state=state, replace=True)
    predict_subsample3 = predict3[target_subsample3.index]
    values3.append(revenue3(target_subsample3, predict_subsample3, 200))
    
values3 = pd.Series(values3)

# Посчитаем среднее значение прибыли
mean3 = values3.mean()
mean3

378354.41642309586

In [99]:
# Посчитаем, сколько точек с отрицательной выручкой
unprofit_values3 = pd.Series(unprofit_values3)
unprofit_values3.count()

87

In [100]:
# А теперь посчитаем положительную выручку
profit_values3 = pd.Series(profit_values3)
profit_values3.count()

913

In [101]:
prof_values3 = pd.Series(prof_values3)
prof_values3.count()

115092

In [91]:
unprof_values3 = pd.Series(unprof_values3)
unprof_values3.count()

84811

In [72]:
confidence_interval3 = (values3.quantile(0.025), values3.quantile(0.975))
confidence_interval3

(-189180.98095071723, 900759.9747676783)

На самом деле - общий вывод не поменялся, все так же рекомендуется использовать регион 2, т.к. он имеет наименее выроятные убытки и наиболее высокую среднюю прибыль.